In [9]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Import data
df_other = pd.read_csv('Other-Implementations/logs/performance_log.csv')
df_metal = pd.read_csv('metal-logs/metal_performance_log.csv')

# Combine both datasets into one DataFrame
df_other['Source'] = 'Other'
df_metal['Source'] = 'Metal'
df_combined = pd.concat([df_other, df_metal], ignore_index=True)
df_combined = df_combined[df_combined['ImageWidth']==256]

# Group by ExecutionType, NumIterations, and Source, and calculate mean for both columns
grouped = df_combined.groupby(['Source', 'ExecutionType', 'NumIterations'])[['ReconstructionTime_ms', 'FinalErrorNorm']].mean()

grouped.reset_index(inplace=True)
grouped = grouped[(grouped['NumIterations'] != 50) & (grouped['NumIterations'] <= 1000)]
df_other = df_other.dropna(subset=['FinalErrorNorm'])
grouped = grouped[grouped['FinalErrorNorm'] > 0]


# Pivot the data to create a single DataFrame
pivoted = grouped.pivot(index='NumIterations', columns=['Source', 'ExecutionType'], values=['ReconstructionTime_ms', 'FinalErrorNorm'])

# Flatten the multi-level columns for easier readability
pivoted.columns = ['_'.join(col).strip() for col in pivoted.columns.values]
pivoted = pivoted.rename_axis('Iterations', axis='index')

pivoted = pivoted.rename(columns={
    'ReconstructionTime_ms_Other_Sequential': 'Seq. (ms)',
    'ReconstructionTime_ms_Other_openmp': 'OMP (ms)',
    'FinalErrorNorm_Other_Sequential': 'Seq. Error Norm',
    'FinalErrorNorm_Other_openmp': 'OMP Error Norm',
    'ReconstructionTime_ms_Metal_Metal': 'Metal (ms)',
    'FinalErrorNorm_Metal_Metal': 'Metal Error Norm'
})

# Define the new column order
new_column_order = [
    'Seq. (ms)',         
    'OMP (ms)',    
    'Metal (ms)',    
    'Seq. Error Norm',     
    'OMP Error Norm',        
    'Metal Error Norm'   
]

# Reorder the columns
pivoted = pivoted[new_column_order]

# Add speedup columns
pivoted['Speedup: OMP vs Seq'] = pivoted['Seq. (ms)'] / pivoted['OMP (ms)']
pivoted['Speedup: Metal vs Seq'] = pivoted['Seq. (ms)'] / pivoted['Metal (ms)']
pivoted['Speedup: Metal vs OMP'] = pivoted['OMP (ms)'] / pivoted['Metal (ms)']

# Calculate the global minimum and maximum across the selected columns
vmin = pivoted[['Speedup: OMP vs Seq', 'Speedup: Metal vs Seq', 'Speedup: Metal vs OMP']].min().min() 
vmax = pivoted[['Speedup: OMP vs Seq', 'Speedup: Metal vs Seq', 'Speedup: Metal vs OMP']].max().max()

# Apply the gradient with consistent vmin and vmax
styled_table = pivoted.style.format(precision=3)\
    .background_gradient(subset=['Speedup: OMP vs Seq', 'Speedup: Metal vs Seq', 'Speedup: Metal vs OMP'], cmap='Blues', vmin=vmin, vmax=vmax)
styled_table



,Seq. (ms),OMP (ms),Metal (ms),Seq. Error Norm,OMP Error Norm,Metal Error Norm,Speedup: OMP vs Seq,Speedup: Metal vs Seq,Speedup: Metal vs OMP
Iterations,,,,,,,,,
1,124.874,71.500,12.129,0.996,0.996,0.996,1.746,10.295,5.895
10,1224.541,732.940,69.416,0.965,0.965,0.965,1.671,17.641,10.559
100,18410.171,7677.612,521.556,0.808,0.808,0.808,2.398,35.299,14.721
500,129114.500,39806.083,2400.600,0.661,0.661,0.661,3.244,53.784,16.582
1000,226751.500,88153.525,4855.343,0.576,0.576,0.576,2.572,46.701,18.156


In [202]:
# Import data
df_other = pd.read_csv('Other-Implementations/logs/normalisation_log.csv')
df_metal = pd.read_csv('metal-logs/normalisation_log.csv')

# Combine both datasets into one DataFrame
df_metal['Source'] = 'Metal'
df_other['Source'] = 'Other'
df_combined = pd.concat([df_metal, df_other], ignore_index=True)

# Group by Type, ImageWidth,NumAngles,NumDetectors
grouped = df_combined.groupby(['Type', 'ImageWidth', 'NumAngles', 'NumDetectors'])[['NormalisationTime']].mean().reset_index()

# Pivot the data to create a single DataFrame
pivoted = grouped.pivot(index=['ImageWidth', 'NumAngles', 'NumDetectors'], columns=['Type'], values='NormalisationTime')
pivoted.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in pivoted.columns.values]
pivoted.reset_index(inplace=True)

# Rename columns for clarity
pivoted = pivoted.rename(columns={
    'ImageWidth': 'Image Size',
    'NumAngles': 'Angles',
    'NumDetectors': 'Detectors',
    'Metal': 'Metal (ms)',
    'Sequential': 'Seq. (ms)',
    'OpenMP': 'OMP (ms)'
})
pivoted['Speedup: Metal vs Seq.'] = pivoted['Seq. (ms)'] / pivoted['Metal (ms)']
pivoted['Speedup: OMP vs Seq.'] = pivoted['Seq. (ms)'] / pivoted['OMP (ms)']
pivoted['Speedup: Metal vs OMP.'] = pivoted['OMP (ms)'] / pivoted['Metal (ms)']
pivoted['Sinogram Size'] = pivoted['Angles'] * pivoted['Detectors']
pivoted = pivoted[['Image Size', 'Angles', 'Detectors', 'Sinogram Size', 'Seq. (ms)','OMP (ms)', 'Metal (ms)', 'Speedup: OMP vs Seq.', 'Speedup: Metal vs Seq.', 'Speedup: Metal vs OMP.']]
styled_table = pivoted.style.format(precision=3)\
    .background_gradient(subset=['Speedup: Metal vs Seq.', 'Speedup: OMP vs Seq.', 'Speedup: Metal vs OMP.'], cmap='Blues')
styled_table
styled_table.to_html('table4.html')


In [203]:
from scipy.stats import zscore
# Import data
df_other = pd.read_csv('Other-Implementations/logs/performance_log.csv')
df_metal = pd.read_csv('metal-logs/metal_performance_log.csv')

# Combine both datasets into one DataFrame
df_metal['Source'] = 'Metal'
df_other['Source'] = 'Other'

# Drop rows with NAN values in ScanTime
df_other = df_other.dropna(subset=['ScanTime'])

def remove_outliers_zscore(df, column, threshold=3):
    z_scores = zscore(df[column])
    return df[abs(z_scores) < threshold]

# Remove outliers for ScanTime in Metal
df_metal = remove_outliers_zscore(df_metal, 'ScanTime')
df_other = remove_outliers_zscore(df_other, 'ScanTime')
df_combined = pd.concat([df_metal, df_other], ignore_index=True)


# Group by Type, ImageWidth,NumAngles,NumDetectors
grouped = df_combined.groupby(['ExecutionType', 'ImageWidth', 'NumAngles', 'NumDetectors'])[['ScanTime']].mean().reset_index()
# Pivot the data to create a single DataFrame
pivoted = grouped.pivot(index=['ImageWidth', 'NumAngles', 'NumDetectors'], columns=['ExecutionType'], values='ScanTime')
pivoted.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in pivoted.columns.values]
pivoted.reset_index(inplace=True)

# Rename columns for clarity
pivoted = pivoted.rename(columns={
    'ImageWidth': 'Image Size',
    'NumAngles': 'Angles',
    'NumDetectors': 'Detectors',
    'Metal': 'Metal Scan (ms)',
    'Sequential': 'Seq. Scan (ms)',
    'openmp': 'OMP Scan (ms)'
})
pivoted['Matrix Nonzero Elements'] = None 

# Add custom values for indices 0, 1, and 2
pivoted.loc[0, 'Matrix Nonzero Elements'] = 18737864  
pivoted.loc[1, 'Matrix Nonzero Elements'] = 74912415  
pivoted.loc[2, 'Matrix Nonzero Elements'] = 299563448
pivoted['Speedup: Metal vs Seq.'] = pivoted['Seq. Scan (ms)'] / pivoted['Metal Scan (ms)']
pivoted['Speedup: OMP vs Seq.'] = pivoted['Seq. Scan (ms)'] / pivoted['OMP Scan (ms)']
pivoted['Speedup: Metal vs OMP'] = pivoted['OMP Scan (ms)'] / pivoted['Metal Scan (ms)']

vmin = pivoted[['Speedup: OMP vs Seq.', 'Speedup: Metal vs Seq.', 'Speedup: Metal vs OMP']].min().min() 
vmax = pivoted[['Speedup: OMP vs Seq.', 'Speedup: Metal vs Seq.', 'Speedup: Metal vs OMP']].max().max()
pivoted = pivoted[['Image Size', 'Angles', 'Detectors', 'Matrix Nonzero Elements', 'Seq. Scan (ms)','OMP Scan (ms)', 'Metal Scan (ms)', 'Speedup: OMP vs Seq.', 'Speedup: Metal vs Seq.', 'Speedup: Metal vs OMP']]
styled_table = pivoted.style.format(precision=3)\
    .background_gradient(subset=['Speedup: Metal vs Seq.', 'Speedup: OMP vs Seq.', 'Speedup: Metal vs OMP'], cmap='Blues', vmin=vmin, vmax=vmax)
styled_table

,Image Size,Angles,Detectors,Matrix Nonzero Elements,Seq. Scan (ms),OMP Scan (ms),Metal Scan (ms),Speedup: OMP vs Seq.,Speedup: Metal vs Seq.,Speedup: Metal vs OMP
0,256,90,725,18737864,58.115,33.145,29.546,1.753,1.967,1.122
1,512,90,1449,74912415,238.268,109.627,64.384,2.173,3.701,1.703
2,1024,90,2897,299563448,1467.300,763.292,412.117,1.922,3.560,1.852
